In [1]:
import geopandas as gpd
import pandas as pd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon

%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time


import networkx as nx


In [2]:
def is_geohash_in_bounding_box(current_geohash, bbox_coordinates):
    """Checks if the box of a geohash is inside the bounding box

    :param current_geohash: a geohash
    :param bbox_coordinates: bounding box coordinates
    :return: true if the the center of the geohash is in the bounding box
    """

    coordinates = geohash.decode(current_geohash)
    geohash_in_bounding_box = (bbox_coordinates[0] < coordinates[0] < bbox_coordinates[2]) and (
            bbox_coordinates[1] < coordinates[1] < bbox_coordinates[3])
    return geohash_in_bounding_box
def build_geohash_box(current_geohash):
    """Returns a GeoJSON Polygon for a given geohash

    :param current_geohash: a geohash
    :return: a list representation of th polygon
    """

    b = geohash.bbox(current_geohash)
    polygon = [(b['w'], b['s']), (b['w'], b['n']), (b['e'], b['n']), (b['e'], b['s'],), (b['w'], b['s'])]
    return polygon
def compute_geohash_tiles(bbox_coordinates,GEOHASH_PRECISION):
    """Computes all geohash tile in the given bounding box

    :param bbox_coordinates: the bounding box coordinates of the geohashes
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding box, assuming the earth is flat ;)
    center_latitude = (bbox_coordinates[0] + bbox_coordinates[2]) / 2
    center_longitude = (bbox_coordinates[1] + bbox_coordinates[3]) / 2

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            if neighbor not in checked_geohashes and is_geohash_in_bounding_box(neighbor, bbox_coordinates):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes
def create_polygon(coordinates):
    polygon_geom = Polygon(coordinates)
    crs = {'init': 'epsg:4326'}
    polygon = gp.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])     
    return polygon.geometry # return polygon['geometry']
# this function is for save the geohashes into a json file 
def write_geohash_layer(geohashes,filename):
    """Writes a grid layer based on the geohashes

    :param geohashes: a list of geohashes
    """

    layer = MultiLineString([build_geohash_box(gh) for gh in geohashes])
    with open('result/' + filename + '.json', 'wb') as f:
        f.write(geojson.dumps(layer,sort_keys=True).encode('utf-8'))

def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    
    #center_latitude = polygon.centroid.coords[0][0]
    #center_longitude = polygon.centroid.coords[0][1]
    
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes

def getNewDistance(lat1,lon1,degree,radius):
    R = 6378.1 #Radius of the Earth
    brng = math.radians(degree) #Bearing is 90 degrees converted to radians.
    d = radius #Distance in km

    #lat2  52.20444 - the lat result I'm hoping for
    #lon2  0.36056 - the long result I'm hoping for.

    lat1 = math.radians(lat1) #Current lat point converted to radians
    lon1 = math.radians(lon1) #Current long point converted to radians

    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1), math.cos(d/R)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    print(lat2)
    print(lon2)
    return lat2,lon2


def getDistance(lat1, long1, distance, bearing):
    
    try:
        # Define starting point.
        start = geopy.Point(lat1, long1)

        # Define a general distance object, initialized with a distance of 1 km.
        d = geopy.distance.VincentyDistance(kilometers = distance)

        # Use the `destination` method with a bearing of 0 degrees (which is north)
        # in order to go from point `start` 1 km to north.
        #print d.destination(point=start, bearing=0)
        return d.destination(point=start,bearing=bearing)
    except:
        pass

def quadraticpolygon(c,a,b):
    try:
        a = list(a)
        b = list(b)
        c = list(c)
        coords = np.array([a,b,c])
        f = scipy.interpolate.interp1d(coords[:, 0], coords[:, 1],kind='quadratic') #, kind='quadratic'
        fine_x = np.linspace(np.min(coords[:, 0]), np.max(coords[:, 0]),20)
        fine_y = f(fine_x)
        fine_coords = zip(fine_x, fine_y)
        #fine_line = LineString(fine_coords)
        myshape = geometry.Polygon(fine_coords)
        return myshape
    except:
        pass

    
def generatePolygon(lat1, long1, radius, angle1, angle2):
    import geopandas as gpd
    from shapely.geometry import Polygon
    lat_point_list = []
    lon_point_list = []
    point1 = getDistance(lat1, long1, radius, angle1)
    point2 = getDistance(lat1, long1, radius, angle2)
    
    point1_lat = point1[0]
    point1_lon = point1[1]
    
    point2_lat = point2[0]
    point2_lon = point2[1]
    
    lat_point_list.append(lat1)
    lat_point_list.append(point1_lat)
    lat_point_list.append(point2_lat)
    
    lon_point_list.append(long1)
    lon_point_list.append(point1_lon)
    lon_point_list.append(point2_lon)
    
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    #print(polygon.geometry)
    
    return polygon.geometry



def compute_geohash_tiles_from_polygon(polygon,GEOHASH_PRECISION):
    """Computes all hex tile in the given polygon

    :param polygon: the polygon
    :return: a list of geohashes
    """

    checked_geohashes = set()
    geohash_stack = set()
    geohashes = []
    # get center of bounding, assuming the earth is flat ;)
    center_latitude = polygon.centroid.coords[0][1]
    center_longitude = polygon.centroid.coords[0][0]

    center_geohash = geohash.encode(center_latitude, center_longitude, precision=GEOHASH_PRECISION)
    geohashes.append(center_geohash)
    geohash_stack.add(center_geohash)
    checked_geohashes.add(center_geohash)
    while len(geohash_stack) > 0:
        current_geohash = geohash_stack.pop()
        neighbors = geohash.neighbors(current_geohash)
        for neighbor in neighbors:
            point = geometry.Point(geohash.decode(neighbor)[::-1])
            if neighbor not in checked_geohashes and polygon.contains(point):
                geohashes.append(neighbor)
                geohash_stack.add(neighbor)
                checked_geohashes.add(neighbor)
    return geohashes


def splitDataFrameList(df,target_column,separator):
    import pandas
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pandas.DataFrame(new_rows)
    return new_df

def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [4]:
sg_road = gp.GeoDataFrame.from_file('singapore_data/singapore_roads.shp')
sg_road.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,4386520,5113,primary,Orchard Road,None,F,50,0,F,F,...,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,"LINESTRING (103.83006 1.30602, 103.83011 1.305..."
1,4887867,5122,residential,Hougang Ave 1,None,B,50,0,F,F,...,Singapore,SGP.4_1,North-East,None,None,Region,Region,None,None,"LINESTRING (103.88738 1.34895, 103.88767 1.348..."
2,8096835,5113,primary,Scotts Road,None,F,60,0,F,F,...,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,"LINESTRING (103.83861 1.31259, 103.83846 1.312..."
3,9584642,5115,tertiary,Keng Lee Road,None,F,50,0,F,F,...,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,"LINESTRING (103.84379 1.31370, 103.84495 1.313..."
4,9584847,5153,footway,None,None,B,0,0,F,F,...,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,"LINESTRING (103.84081 1.31468, 103.84126 1.313..."


In [6]:
sg_poi = gp.GeoDataFrame.from_file('singapore_data/singapore_poi.shp')
sg_poi.head()

,osm_id,code,fclass,name,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,26778938,2602,atm,None,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.85070 1.30475)
1,206529415,2502,bakery,Yio Chu Kang,SGP,Singapore,SGP.4_1,North-East,None,None,Region,Region,None,None,MULTIPOINT (103.84494 1.38179)
2,232105741,2742,viewpoint,None,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.85852 1.30197)
3,233497138,2602,atm,None,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.84871 1.29858)
4,246801928,2721,attraction,Mural,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.83833 1.27653)


In [7]:
## Getting the centroid coordinates for lat long
sg_road['lat'] = sg_road['geometry'].centroid.y
sg_road['lon'] = sg_road['geometry'].centroid.x

sg_poi['lat'] = sg_poi['geometry'].centroid.y
sg_poi['lon'] = sg_poi['geometry'].centroid.x

In [29]:
sg_road['geohash'] = sg_road['geometry'].apply(lambda x:compute_geohash_tiles_from_polygon(x,7))
sg_poi['geohash'] = sg_poi['geometry'].apply(lambda x:compute_geohash_tiles_from_polygon(x,7))

In [56]:
poi_explode = explode(sg_poi, ['geohash'])

In [57]:
poi_explode.head()

,CC_1,ENGTYPE_1,GID_0,GID_1,HASC_1,NAME_0,NAME_1,NL_NAME_1,TYPE_1,VARNAME_1,code,fclass,geometry,lat,lon,name,osm_id,geohash
0,None,Region,SGP,SGP.1_1,None,Singapore,Central,None,Region,None,2602,atm,MULTIPOINT (103.85070 1.30475),1.304752,103.850697,None,26778938,w21z7jt
1,None,Region,SGP,SGP.4_1,None,Singapore,North-East,None,Region,None,2502,bakery,MULTIPOINT (103.84494 1.38179),1.381790,103.844936,Yio Chu Kang,206529415,w21zg59
2,None,Region,SGP,SGP.1_1,None,Singapore,Central,None,Region,None,2742,viewpoint,MULTIPOINT (103.85852 1.30197),1.301971,103.858523,None,232105741,w21z7m5
3,None,Region,SGP,SGP.1_1,None,Singapore,Central,None,Region,None,2602,atm,MULTIPOINT (103.84871 1.29858),1.298583,103.848711,None,233497138,w21z7hk
4,None,Region,SGP,SGP.1_1,None,Singapore,Central,None,Region,None,2721,attraction,MULTIPOINT (103.83833 1.27653),1.276526,103.838326,Mural,246801928,w21z6bk


In [58]:
poi_geohash = poi_explode.groupby('geohash', as_index=False).count()
poi_geohash.sort_values(by='osm_id',ascending=False).head(15)

,geohash,CC_1,ENGTYPE_1,GID_0,GID_1,HASC_1,NAME_0,NAME_1,NL_NAME_1,TYPE_1,VARNAME_1,code,fclass,geometry,lat,lon,name,osm_id
987,w21z75j,0,84,84,84,0,84,84,0,84,0,84,84,84,84,84,83,84
843,w21z6yv,0,80,80,80,0,80,80,0,80,0,80,80,80,80,80,70,80
3379,w23bh25,0,79,79,79,0,79,79,0,79,0,79,79,79,79,79,65,79
1110,w21z7k8,0,78,78,78,0,78,78,0,78,0,78,78,78,78,78,74,78
2929,w21zujq,0,65,65,65,0,65,65,0,65,0,65,65,65,65,65,62,65
2132,w21zetv,0,64,64,64,0,64,64,0,64,0,64,64,64,64,64,62,64
375,w21z3yt,0,61,61,61,0,61,61,0,61,0,61,61,61,61,61,56,61
1111,w21z7k9,0,60,60,60,0,60,60,0,60,0,60,60,60,60,60,56,60
883,w21z713,0,54,54,54,0,54,54,0,54,0,54,54,54,54,54,50,54
1117,w21z7kg,0,54,54,54,0,54,54,0,54,0,54,54,54,54,54,40,54


In [59]:
sg_poi_cumulative = poi_geohash[['geohash', 'osm_id']]
sg_poi_cumulative.columns = ['geohash','no_poi']
sg_poi['geohash'] = sg_poi['geohash'].astype(str)

In [60]:
sg_poi_cumulative.head()

,geohash,no_poi
0,w21xjfc,1
1,w21xjzu,1
2,w21xnyn,3
3,w21xpnd,1
4,w21xpye,1


In [33]:
sg_road['geohash'] = sg_road['geohash'].astype(str)
sg_road['geohash'] = sg_road['geohash'].str[1:]
sg_road['geohash'] = sg_road['geohash'].str[:-1]
sg_road['geohash'] = sg_road['geohash'].str[1:]
sg_road['geohash'] = sg_road['geohash'].str[:-1]

In [34]:
sg_road.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,...,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,lat,lon,geohash
0,4386520,5113,primary,Orchard Road,None,F,50,0,F,F,...,None,None,Region,Region,None,None,"LINESTRING (103.83006 1.30602, 103.83011 1.305...",1.305964,103.830166,w21z6tw
1,4887867,5122,residential,Hougang Ave 1,None,B,50,0,F,F,...,None,None,Region,Region,None,None,"LINESTRING (103.88738 1.34895, 103.88767 1.348...",1.348962,103.887582,w21zsj8
2,8096835,5113,primary,Scotts Road,None,F,60,0,F,F,...,None,None,Region,Region,None,None,"LINESTRING (103.83861 1.31259, 103.83846 1.312...",1.312089,103.837838,w21z6yu
3,9584642,5115,tertiary,Keng Lee Road,None,F,50,0,F,F,...,None,None,Region,Region,None,None,"LINESTRING (103.84379 1.31370, 103.84495 1.313...",1.313793,103.844403,w21z7p1
4,9584847,5153,footway,None,None,B,0,0,F,F,...,None,None,Region,Region,None,None,"LINESTRING (103.84081 1.31468, 103.84126 1.313...",1.314032,103.841054,w21z6zn


In [24]:
unique(sg_road['fclass'])

array(['bridleway', 'cycleway', 'footway', 'living_street', 'motorway',
       'motorway_link', 'path', 'pedestrian', 'primary', 'primary_link',
       'residential', 'secondary', 'secondary_link', 'service', 'steps',
       'tertiary', 'tertiary_link', 'track', 'track_grade1',
       'track_grade2', 'track_grade3', 'track_grade4', 'track_grade5',
       'trunk', 'trunk_link', 'unclassified'], dtype=object)

In [25]:
unique(sg_road['maxspeed'])

array([ 0,  5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90], dtype=int64)

In [26]:
len(sg_road)

175212

In [37]:
sg_road.columns

Index(['osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed',
       'layer', 'bridge', 'tunnel', 'GID_0', 'NAME_0', 'GID_1', 'NAME_1',
       'VARNAME_1', 'NL_NAME_1', 'TYPE_1', 'ENGTYPE_1', 'CC_1', 'HASC_1',
       'geometry', 'lat', 'lon', 'geohash'],
      dtype='object')

In [42]:
sg_road_speed = sg_road[['maxspeed','geohash']]

In [43]:
sg_road_speed.head()

,maxspeed,geohash
0,50,w21z6tw
1,50,w21zsj8
2,60,w21z6yu
3,50,w21z7p1
4,0,w21z6zn


In [48]:
sg_road_speed = sg_road_speed[sg_road_speed['maxspeed']!= 0]

In [49]:
sg_road_speed.head()

,maxspeed,geohash
0,50,w21z6tw
1,50,w21zsj8
2,60,w21z6yu
3,50,w21z7p1
5,60,w21z6zj


In [52]:
sg_road_speed.groupby('geohash').mean().reset_index()

,geohash,maxspeed
0,w21xjbe,50.0
1,w21xjc6,50.0
2,w21xjcd,50.0
3,w21xjcf,50.0
4,w21xjf1,70.0
...,...,...
13375,w23bp22,50.0
13376,w23bp23,50.0
13377,w23bp24,50.0
13378,w23bp28,50.0


In [53]:
sg_road_speed['maxspeed'].mean()

52.24763332388013

In [55]:
sg_poi.head()

,osm_id,code,fclass,name,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry,lat,lon,geohash
0,26778938,2602,atm,None,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.85070 1.30475),1.304752,103.850697,[w21z7jt]
1,206529415,2502,bakery,Yio Chu Kang,SGP,Singapore,SGP.4_1,North-East,None,None,Region,Region,None,None,MULTIPOINT (103.84494 1.38179),1.381790,103.844936,[w21zg59]
2,232105741,2742,viewpoint,None,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.85852 1.30197),1.301971,103.858523,[w21z7m5]
3,233497138,2602,atm,None,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.84871 1.29858),1.298583,103.848711,[w21z7hk]
4,246801928,2721,attraction,Mural,SGP,Singapore,SGP.1_1,Central,None,None,Region,Region,None,None,MULTIPOINT (103.83833 1.27653),1.276526,103.838326,[w21z6bk]


In [61]:
singapore_geofabrik = pd.merge(sg_poi_cumulative, sg_road_speed, how='outer', on='geohash')

In [63]:
singapore_geofabrik.head()

,geohash,no_poi,maxspeed
0,w21xjfc,1.0,70.0
1,w21xjzu,1.0,NaN
2,w21xnyn,3.0,NaN
3,w21xpnd,1.0,NaN
4,w21xpye,1.0,NaN


In [64]:
singapore_geofabrik.to_csv('sg_geofabrik.csv', index=False)